In [3]:
%load_ext autoreload
%autoreload 2

In [21]:
import os
import sys
import requests
import time
import math
import hashlib
from lxml import html
import pickle
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 

sys.setrecursionlimit(25000)
# sys.path.append('../python')
# import monitor_url as murl

In [5]:
df = pd.read_csv('../datasets/compiled_company_info.csv')
df = df.drop_duplicates()
df.head(len(df))

,Company,Yahoo Listed Co.,Symbol,Exchange,Market Cap,Company Size,Is American,Home URL,Press Release URL
0,AstraZeneca,AstraZeneca PLC,AZN,NYSE,"$140,554,649,600",v-large,Y,http://www.astrazeneca.com,https://www.astrazeneca.com/media-centre/press...
1,CanSino Biological Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Beijing Institute of Biotechnology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Moderna,"Moderna, Inc.",MRNA,NASDAQ,"$21,742,589,952",medium,Y,http://www.modernatx.com,https://investors.modernatx.com/news-releases/
4,NIAID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Wuhan Institute of Biological Products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Sinopharm,"Sinopharm Group Co., Ltd.",SHTDY,OTC Markets,"$8,092,427,776",small,N,http://www.sinopharmgroup.com.cn,http://ir.sinopharmgroup.com.cn/
7,Beijing Institute of Biological Products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Sinovac,Sinovac Biotech Ltd.,SVA,NASDAQ,"$460,249,280",small,Y,http://www.sinovacbio.com,http://www.sinovacbio.com/?optionid=754
9,Novavax,"Novavax, Inc.",NVAX,NASDAQ,"$4,791,271,936",small,Y,http://www.novavax.com,https://ir.novavax.com/press-releases


# Monitoring for changes
~~ As a starting point, looking into leveraging the URLWATCH package:
https://github.com/thp/urlwatch ~~

~~ To do from scratch, here are some references: ~~
- ~~https://chrisalbon.com/python/web_scraping/monitor_a_website/ ~~
- ~~https://www.adventuresintechland.com/detect-when-a-webpage-changes-with-python/~~
- ~~https://www.codementor.io/@gergelykovcs/how-and-why-i-built-a-simple-web-scrapig-script-to-notify-us-about-our-favourite-food-fcrhuhn45 ~~

Went the BeautifulSoup route

In [41]:
page = requests.get('https://investors.biontech.de/press-releases/').text
soup = BeautifulSoup(page, 'html.parser')
body = soup.find('body')
print(body)

<body class="body-sidebars-none nir-node nir-node--type-nir-landing-page nir-node--5816 path-node page-node-type-nir-landing-page">
<div id="skip">
<a class="visually-hidden focusable skip-link" href="#main-menu">
        Skip to main navigation
      </a>
</div>
<div class="dialog-off-canvas-main-canvas" data-off-canvas-main-canvas="">
<div class="body-page">
<header class="header-v1" id="header">
<div class="topbar">
<div class="topbar-inner">
<div class="container">
<div class="content">
<h2 class="visually-hidden">Topbar Nav</h2>
<nav aria-labelledby="block-topbarmenu-menu" class="topBarMenu block block-menu navigation menu--top-bar-menu" id="block-topbarmenu" role="navigation">
<div class="block-content">
<ul class="gva_menu">
<li class="menu-item menu-item--active-trail">
<a class="is-active" href="/investors-media" target="" title="Investors &amp; Media">Investors &amp; Media</a>
</li>
<li class="menu-item">
<a href="https://biontech.de/careers" target="" title="Careers">Careers

In [77]:
# Testing with local html files
pg_original = '../datasets/original_webpage.html'
pg_updated = '../datasets/updated_webpage.html'

soup_original = BeautifulSoup(open(pg_original), 'html.parser')
soup_updated = BeautifulSoup(open(pg_updated), 'html.parser')


body_original = soup_original.find('body')
body_updated = soup_updated.find('body')

links_original = list(body_original.find_all('a'))
links_updated = list(body_updated.find_all('a'))

# print(f'Originally: {len(links_original)} links \n\
# Href: {len(links_href)} links \n')

# Method from
# https://stackoverflow.com/questions/41125909/python-find-elements-in-one-list-that-are-not-in-the-other
links_diff = list(set(links_updated) - set(links_original))

for link in links_diff:
    print(link)
    ref = link.get('href')
    content = link.text
    
    print(f'{content}: {ref} \n')
    #print(f'{link.get('href')}\n')

<a href="/node/8019/pdf" target="_blank"><span class="file-icon widget-file--application-pdf"></span></a>
: /node/8019/pdf 

<a class="skip-me" href="/news-releases/news-release-details/independence-day-phase-test-positive">Independence Day Phase Test Positive</a>
Independence Day Phase Test Positive: /news-releases/news-release-details/independence-day-phase-test-positive 



In [45]:
def url_to_filename(base_url:str):
    base_url = prune_url(base_url)
    
    forbidden_ascii = ['/', '\\', '\|', ':', '?',
                      '\'', '\"', '?', '*', '>', '<']
    
    for ascii_char in forbidden_ascii:
        base_url = base_url.replace(ascii_char, '-')
    
    base_url = base_url.replace('.', '_')
    
    return base_url
    
    
def prune_url(full_url:str):
    cut_chars = ['https://', 'http://', 'www.']
    
    for cut in cut_chars:
        full_url = full_url.replace(cut, '')
    
    if full_url[-1] == '/':
        pruned_url = full_url[0:-1]
    else:
        pruned_url = full_url
    
    return pruned_url





def load_cache(base_url, path='logs/'):
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.html'
    f=open(filename, 'r')
    data_cache = f.read()
    return data_cache


def store_cache(base_url, data, path='logs/'):
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.html'
    f = open(filename, 'w')
    f.write(data)
    f.close()
                      

def cache_updated(base_url, data, path='logs/'):
    # Check cache exists
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.html'
    cache_exists = os.path.isfile(filename)
    
    if cache_exists:
        data_cache = load_cache(base_url, path=path)
        f=open(filename, 'r')
        data_cache = f.read()
        
        soup_cache = BeautifulSoup(data_cache, 'html.parser')
        body_cache = soup_cache.find_all('body')
        soup_data = BeautifulSoup(data, 'html.parser')
        body_data = soup_data.find_all('body')
        
        hash_cache = hash(str(body_cache))
        hash_data = hash(str(body_data))
        
        
        if body_cache == body_data:
            message = f'No update from {base_url} \n'
            write_log(message, base_url, path=path)
            return False
        else:
            message = f'Update detected from {base_url} \n'
            write_log(message, base_url, path=path)
            return True
        
    else:
        message1 = f'No cached webpage found for {base_url} \n'
        message2 = f'Initializing page in {filename} \n'
        write_log(f'{message1}{message2}', base_url, path=path)
        store_cache(base_url, data, path=path)
        return False
        
        
        
        

def load_scache(base_url, path='logs/'):
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.pkl'
    f=open(filename, 'rb')
    serial_data = f.read()
    data_cache1 = pickle.loads(serial_data)
    data_cache = pickle.loads(data_cache1) # Need to revisit
    return data_cache


def store_scache(base_url, data, path='logs/'):
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.pkl'
    f = open(filename, 'wb')
    serial_data = pickle.dumps(data)
    pickle.dump(serial_data, f)
    # f.write(data_ser)
    f.close()
                      

def scache_updated(base_url, data, path='logs/'):
    # Check cache exists
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.pkl'
    cache_exists = os.path.isfile(filename)
    
    if cache_exists:
        data_cache = load_cache(base_url, path=path)
        f=open(filename, 'rb')
        serial_cache = f.read()
        serial_data = pickle.dumps(data)
        
        if serial_data == serial_cache:
            message = f'No update from {base_url} \n'
            write_log(message, base_url, path=path)
            return False
        else:
            message = f'Update detected from {base_url} \n'
            write_log(message, base_url, path=path)
            return True
        
    else:
        message1 = f'No cached webpage found for {base_url} \n'
        message2 = f'Initializing page in {filename} \n'
        write_log(f'{message1}{message2}', base_url, path=path)
        store_cache(base_url, data, path=path)
        return False
        

def write_log(message:str, base_url, path='logs/'):
    print(message)
    url_filename = url_to_filename(base_url)
    filename = f'{path}LOG-{url_filename}.txt'
    f = open(filename, 'w')
    f.write(message)
    f.close()
    
def get_links(html_data):
    anchors = get_anchors(html_data)
    hrefs, content = parse_anchors(anchors)
    
    return hrefs, content, anchors
    

def get_anchors(html_data):
    if html_data.find('body') == None:
        html_body = html_data
    else:
        html_body = html_data.find('body')   
        
    anchors = list(html_body.find_all('a'))
    
    return anchors


def parse_anchors(anchors):
    hrefs = []
    content = []
    
    for anchor in anchors:
        hrefs.append(anchor.get('href'))
        content.append(anchor.text)
    
    return hrefs, content

def get_new_diff(new_data, old_data):
    diff = list(set(new_data) - set(old_data))
    return diff


def is_na(subject):
    if isinstance(subject, str):
        na_versions = ['n/a', 'nan']
        if subject.lower() in na_versions:
            return True
        else: 
            return False
    elif isinstance(subject, float):
        if math.isnan(subject):
            return True
    elif isinstance(subject, bool):
        return subject
    else:
        return False
        
    
        

In [46]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

for ii in range(len(df)): 
    co, yco, sym, exch, mkcap, size, am, url_home, url_pr = \
    df.iloc[ii].T.values
    
    if not is_na(url_pr):
        try:
            r = requests.get(url_pr, headers=headers)
        except:
            message = f'CONNECTION FAILED! - Status Code: {r.status_code}\n'
            print(f'\n{message}')
            write_log(message, url_pr)
            continue
            
        page = r.text
        soup = BeautifulSoup(page, 'html.parser')
        body = soup.find('body')
        
        if not body:
            continue
            
        anchors = get_anchors(body)
        
        has_update = cache_updated(url_pr, page)
        print(has_update)
        print('\n')
     
        if has_update:
            old_anchors = load_cache(url_pr)
            # old_anchors = get_anchors(old_body)
            # new_anchors = get_anchors(body)
            diff_anchors = get_new_diff(anchors, old_anchors)
            diff_hrefs, diff_contents = parse_anchors(diff_anchors)
            
            if len(diff_anchors) > 0:
                message = f'New links found: \n---------------'
                write_log(message, url_pr)
                
                for diff_href, diff_content in zip(diff_hrefs, diff_contents):
                    message = f'{diff_href}  ::  {diff_content}'
                    write_log(message, url_pr)
                    
            else:
                message = 'Updated detected but no new anchors'
                write_log(message, url_pr)
                
            
            store_cache(url_pr, page)
            print('\n\n')
    
    else:
        print('\n----------------------------------')
        print(f'Skipping {co}')
        print('\n----------------------------------\n\n')
            

No update from https://www.astrazeneca.com/media-centre/press-releases.html 

False



----------------------------------
Skipping CanSino Biological Inc.

----------------------------------



----------------------------------
Skipping Beijing Institute of Biotechnology

----------------------------------


Update detected from https://investors.modernatx.com/news-releases/ 

True


New links found: 
---------------
/news-releases/news-release-details/moderna-highlights-opportunity-mrna-vaccines-its-first-vaccines  ::  Moderna Highlights Opportunity of mRNA Vaccines at its First Vaccines Day
https://www.modernatx.com/mrna-technology/research-engine  ::  Research Engine
/news-releases/news-release-details/moderna-provides-business-updates-and-reports-third-quarter-2019  ::  Moderna Provides Business Updates and Reports Third Quarter 2019 Financial Results
/  ::  
/news-releases/news-release-details/moderna-present-preclinical-data-2019-american-society-gene-cell  ::  Moderna to Presen


----------------------------------
Skipping Beijing Institute of Biological Products

----------------------------------


No update from http://www.sinovacbio.com/?optionid=754 

False


Update detected from https://ir.novavax.com/press-releases 

True


New links found: 
---------------
/news-releases/news-release-details/novavax-present-ladenburg-thalmann-2019-healthcare-conference  ::  Novavax to Present at the Ladenburg Thalmann 2019 Healthcare Conference
/news-releases/news-release-details/novavax-initiates-pre-clinical-testing-intranasal-influenza  ::  Novavax Initiates Pre-Clinical Testing of Intranasal Influenza Vaccine
/news-releases/news-release-details/novavax-present-rodman-renshaw-techvest-7th-annual-healthcare  ::  Novavax to Present at Rodman & Renshaw Techvest 7th Annual Healthcare Conference
/news-releases/news-release-details/novavax-initiates-phase-2-clinical-trial-rsv-vaccine-candidate  ::  Novavax Initiates Phase 2 Clinical Trial of RSV Vaccine Candidate
/news-re

/news-releases/news-release-details/novavax-presents-positive-clinical-results-world-health  ::  NOVAVAX Presents Positive Clinical Results at The World Health Organization Conference, Geneva, Switzerland
/news-releases/news-release-details/novavax-reports-positive-results-phase-i-trial-respiratory  ::  NOVAVAX Reports Positive Results From Phase I Trial of Respiratory Syncytial Virus Vaccine Candidate
/news-releases/news-release-details/novavax-highlight-rsv-program-data-two-upcoming-meetings  ::  Novavax to Highlight RSV Program Data at Two Upcoming Meetings
/news-releases/news-release-details/novavax-provides-updates-global-pathways-licensure-resvaxtm  ::  Novavax Provides Updates on the Global Pathways to Licensure for ResVax™
/news-releases/news-release-details/icaac-presentations-highlight-potential-protective-effects  ::  ICAAC Presentations Highlight Potential Protective Effects of Novavax' RSV Vaccine Candidate
/news-releases/news-release-details/novavax-announces-management-p

No update from https://investors.biontech.de/press-releases/ 

False



CONNECTION FAILED! - Status Code: 200

CONNECTION FAILED! - Status Code: 200

No cached webpage found for https://www.pfizer.com/news/press-release/press-releases-archive 
Initializing page in logs/CACHE-pfizer_com-news-press-release-press-releases-archive.html 

False


No cached webpage found for http://ir.inovio.com/news-releases/default.aspx 
Initializing page in logs/CACHE-ir_inovio_com-news-releases-default_aspx.html 

False



----------------------------------
Skipping Genexine Consortium

----------------------------------



----------------------------------
Skipping Gamaleya Research Institute

----------------------------------



----------------------------------
Skipping Clover Biopharmaceuticals Inc.

----------------------------------


No cached webpage found for https://www.gsk.com/en-gb/media/press-releases/ 
Initializing page in logs/CACHE-gsk_com-en-gb-media-press-releases.html 

False


No c

In [42]:
# print(url_pr)
#page = requests.get(url_pr, headers=headers)
#print(page.text)
# print(body)
# print(len(body.find_all('a')))
# store_cache(url_pr, page)

# soup_ = BeautifulSoup(page, 'html.parser')
# body_ = soup_.find_all('a')
# print(hash(str(body_)))


# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# page = requests.get(url_pr, headers = headers)
# print(page)
# soup = BeautifulSoup(page.text, 'html.parser')
# body = soup.find_all('body')
# print(body)

# print(soup.prettify())
# cached = load_cache(url_pr)
# print(cached)

# url_filename = url_to_filename(url_pr)
# filename = f'logs/CACHE-{url_filename}.pkl'
# f=open(filename, 'rb')
# serial_data = f.read()
# data_cached = pickle.loads(serial_data)
# cached = pickle.loads(data_cached)
# print(data_cached)

https://investors.modernatx.com/news-releases/
